In [1]:
import numpy as np
import torch
import torch.nn as nn
import os
import nltk
from nltk.tokenize import word_tokenize
from torch.utils.data import Dataset

In [2]:
Poem = []
with open('ferdousi.txt', 'r', encoding = 'utf8') as f:
    for _ in range(99218):
        Poem.append(f.readline())

Poem_Modif = [p[0:-1] for p in Poem[2:]]


Dict = []
Len_Verse = []
for p in Poem_Modif:
    w_token = word_tokenize(p)
    Dict.extend(w_token)
    Len_Verse.append(len(w_token))
Dict = list(set(Dict))
Add1 = ['<شروع>', '<خالی>', '.']
Add1.extend(Dict)
Dict = Add1

print(f'Number of distict words in Shahnameh: {len(Dict)-2}')

Number of distict words in Shahnameh: 18008


In [3]:
# Word to Index
Pin = Poem_Modif[0]
Pin = word_tokenize(Pin)
if len(Pin) < 11:
    for _ in range(11 - len(Pin)):
        Pin.append('<خالی>')
string = '<شروع>'
Pin.insert(0, string)
Pin.insert(len(Pin), '.')
Pin_idx = [Dict.index(p) for p in Pin]
print(Pin)
print(Pin_idx)
#print(word_tokenize(Poem_Modif[0]), Len_Verse[0])

['<شروع>', 'به', 'نام', 'خداوند', 'جان', 'و', 'خرد', '<خالی>', '<خالی>', '<خالی>', '<خالی>', '<خالی>', '.']
[0, 595, 10732, 1680, 1647, 6476, 1052, 1, 1, 1, 1, 1, 2]


In [4]:
class MyDataset(Dataset):
    def __init__(self, Poem, Dictionary, Len_Verse):
        self.poem = Poem
        self.Dict = Dictionary
        self.Lverse = Len_Verse
        self.Lmax = max(self.Lverse)
    def __len__(self):
        return int(len(self.Lverse)/2)
    def __getitem__(self, idx):
        Pin = self.poem[2*idx]
        Pin = word_tokenize(Pin)
        if len(Pin) < self.Lmax:
            for _ in range(self.Lmax - len(Pin)):
                Pin.append('<خالی>')
        string = '<شروع>'
        Pin.insert(0, string)
        Pin.insert(len(Pin), '.')
        Pin_idx = [self.Dict.index(p) for p in Pin]
        
        Pout = self.poem[2*idx + 1]
        Pout = word_tokenize(Pout)
        if len(Pout) < self.Lmax:
            for _ in range(self.Lmax - len(Pout)):
                Pout.append('<خالی>')
        string = '<شروع>'
        Pout.insert(0, string)
        Pout.insert(len(Pout), '.')
        Pout_idx = [self.Dict.index(p) for p in Pout]
        return torch.LongTensor(Pin_idx), torch.LongTensor(Pout_idx)

In [16]:
class RNN_Model(nn.Module):
    def __init__(self, LDict, Lverse):
        super(RNN_Model, self).__init__()
        self.embed_dim = 200
        self.hid_size = 500
        self.Lverse = Lverse
        self.embedding = nn.Embedding(num_embeddings= LDict, embedding_dim= self.embed_dim)
        self.lstm = nn.LSTM(input_size = self.embed_dim, num_layers = 2, hidden_size = self.hid_size,\
                                   batch_first = True, bidirectional = False)
        self.linear = nn.Linear(in_features = 1*self.hid_size, out_features= LDict)
    def forward(self, X):
        word_idx = torch.cat((X[0][:, 1:], X[1][:, 0:-1]), dim = -1)
        word_embed = self.embedding(word_idx)
        hout, _ = self.lstm(word_embed)
        logits = self.linear(hout)
        return logits[:, self.Lverse+1:, :]

class GRU_Model(nn.Module):
    def __init__(self, LDict, Lverse):
        super(GRU_Model, self).__init__()
        self.embed_dim = 200
        self.hid_size = 500
        self.Lverse = Lverse
        self.embedding = nn.Embedding(num_embeddings= LDict, embedding_dim= self.embed_dim)
        self.gru = nn.GRU(input_size = self.embed_dim, num_layers = 2, hidden_size = self.hid_size,\
                                   batch_first = True, bidirectional = False)
        self.linear = nn.Linear(in_features = 1*self.hid_size, out_features= LDict)
    def forward(self, X):
        word_idx = torch.cat((X[0][:, 1:], X[1][:, 0:-1]), dim = -1)
        word_embed = self.embedding(word_idx)
        hout, _ = self.gru(word_embed)
        logits = self.linear(hout)
        return logits[:, self.Lverse+1:, :]
        

In [13]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter

All_Data = MyDataset(Poem_Modif, Dict, Len_Verse);
SplitData = random_split(All_Data, [0.80, 0.20])
Train = SplitData[0]
Test = SplitData[1]
print(len(All_Data), len(Train), len(Test))
batch_size = 64
train_dataloader = DataLoader(Train, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(Test, batch_size = batch_size, shuffle = True)

49608 39687 9921


In [39]:
model_lstm = RNN_Model(len(Dict), max(Len_Verse))
device = 'cuda' if torch.cuda.is_available else 'cpu'
model_lstm.to(device)
optimizer_lstm = torch.optim.Adam(model_lstm.parameters(), lr = 1e-3)
loss_func = nn.CrossEntropyLoss()
writer_lstm = SummaryWriter()

In [18]:
model_gru = GRU_Model(len(Dict), max(Len_Verse))
model_gru.to(device)
optimizer_gru = torch.optim.Adam(model_gru.parameters(), lr = 1e-3)
loss_func = nn.CrossEntropyLoss()
writer_gru = SummaryWriter()

In [19]:
def train_loop(Model, Optimizer, Loss_Func, dataloader, writer, kep, model_name):
    LD = len(dataloader.dataset)
    for i, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        ypred = Model([X, y])
        ypred = ypred.permute(0, 2, 1)
        yy = y[:, 1:]
        loss = Loss_Func(ypred, yy)
        Optimizer.zero_grad()
        loss.backward()
        Optimizer.step()
        if i % 50 == 0:
            print(f'loss = {loss.item():5f}     [{i*batch_size:5}\{LD:5}]')
            writer.add_scalar(f'Loss/train {model_name}', loss, i + kep*len(dataloader))

In [40]:
for k in range(20):
    print(f'\nEpoch: {k}-----------------------------------------')
    model_lstm.train()
    train_loop(model_lstm, optimizer_lstm, loss_func, train_dataloader, writer_lstm, k, 'LSTM')
print('Done!')


Epoch: 0-----------------------------------------
loss = 9.804275     [    0\39687]
loss = 3.911157     [ 3200\39687]
loss = 3.683077     [ 6400\39687]
loss = 3.357572     [ 9600\39687]
loss = 3.469652     [12800\39687]
loss = 3.357467     [16000\39687]
loss = 3.294295     [19200\39687]
loss = 3.328545     [22400\39687]
loss = 3.275915     [25600\39687]
loss = 3.345386     [28800\39687]
loss = 3.214770     [32000\39687]
loss = 3.223907     [35200\39687]
loss = 3.211159     [38400\39687]

Epoch: 1-----------------------------------------
loss = 3.180510     [    0\39687]
loss = 3.116836     [ 3200\39687]
loss = 3.072455     [ 6400\39687]
loss = 3.076463     [ 9600\39687]
loss = 2.968444     [12800\39687]
loss = 2.936721     [16000\39687]
loss = 2.965333     [19200\39687]
loss = 3.036682     [22400\39687]
loss = 2.998941     [25600\39687]
loss = 2.886612     [28800\39687]
loss = 2.891507     [32000\39687]
loss = 3.082515     [35200\39687]
loss = 2.977462     [38400\39687]

Epoch: 2-----

loss = 0.652661     [25600\39687]
loss = 0.705678     [28800\39687]
loss = 0.711845     [32000\39687]
loss = 0.692725     [35200\39687]
loss = 0.739972     [38400\39687]

Epoch: 17-----------------------------------------
loss = 0.517230     [    0\39687]
loss = 0.496532     [ 3200\39687]
loss = 0.595971     [ 6400\39687]
loss = 0.506773     [ 9600\39687]
loss = 0.539782     [12800\39687]
loss = 0.560034     [16000\39687]
loss = 0.587169     [19200\39687]
loss = 0.548896     [22400\39687]
loss = 0.619668     [25600\39687]
loss = 0.579098     [28800\39687]
loss = 0.616159     [32000\39687]
loss = 0.629029     [35200\39687]
loss = 0.597161     [38400\39687]

Epoch: 18-----------------------------------------
loss = 0.431959     [    0\39687]
loss = 0.432603     [ 3200\39687]
loss = 0.438615     [ 6400\39687]
loss = 0.436540     [ 9600\39687]
loss = 0.458717     [12800\39687]
loss = 0.473055     [16000\39687]
loss = 0.511867     [19200\39687]
loss = 0.511793     [22400\39687]
loss = 0.470

In [20]:
for k in range(20):
    print(f'\nEpoch: {k}-----------------------------------------')
    model_gru.train()
    train_loop(model_gru, optimizer_gru, loss_func, train_dataloader, writer_gru, k, 'GRU')
print('Done!')


Epoch: 0-----------------------------------------
loss = 9.797395     [    0\39687]
loss = 3.745973     [ 3200\39687]
loss = 3.566846     [ 6400\39687]
loss = 3.625413     [ 9600\39687]
loss = 3.370488     [12800\39687]
loss = 3.228398     [16000\39687]
loss = 3.447102     [19200\39687]
loss = 3.175660     [22400\39687]
loss = 3.125396     [25600\39687]
loss = 3.242763     [28800\39687]
loss = 3.100804     [32000\39687]
loss = 3.239866     [35200\39687]
loss = 3.004937     [38400\39687]

Epoch: 1-----------------------------------------
loss = 2.773254     [    0\39687]
loss = 2.723202     [ 3200\39687]
loss = 2.764926     [ 6400\39687]
loss = 2.839546     [ 9600\39687]
loss = 2.843634     [12800\39687]
loss = 2.833717     [16000\39687]
loss = 2.663848     [19200\39687]
loss = 2.602902     [22400\39687]
loss = 2.642845     [25600\39687]
loss = 2.800781     [28800\39687]
loss = 2.660796     [32000\39687]
loss = 2.517396     [35200\39687]
loss = 2.749510     [38400\39687]

Epoch: 2-----

loss = 0.122340     [25600\39687]
loss = 0.102624     [28800\39687]
loss = 0.175801     [32000\39687]
loss = 0.136267     [35200\39687]
loss = 0.167777     [38400\39687]

Epoch: 17-----------------------------------------
loss = 0.058874     [    0\39687]
loss = 0.082924     [ 3200\39687]
loss = 0.106420     [ 6400\39687]
loss = 0.118103     [ 9600\39687]
loss = 0.093098     [12800\39687]
loss = 0.085960     [16000\39687]
loss = 0.077900     [19200\39687]
loss = 0.106544     [22400\39687]
loss = 0.125715     [25600\39687]
loss = 0.148488     [28800\39687]
loss = 0.149465     [32000\39687]
loss = 0.144113     [35200\39687]
loss = 0.159174     [38400\39687]

Epoch: 18-----------------------------------------
loss = 0.095960     [    0\39687]
loss = 0.070018     [ 3200\39687]
loss = 0.088007     [ 6400\39687]
loss = 0.069357     [ 9600\39687]
loss = 0.104755     [12800\39687]
loss = 0.101573     [16000\39687]
loss = 0.068733     [19200\39687]
loss = 0.105083     [22400\39687]
loss = 0.126

In [41]:
writer_gru.flush()
writer_gru.close() 
writer_lstm.flush()
writer_lstm.close() 
#%load_ext tensorboard
%tensorboard --logdir='runs' --port=8892 

Reusing TensorBoard on port 8892 (pid 4756), started 1:54:59 ago. (Use '!kill 4756' to kill it.)

In [29]:
def Generate_Verse(model, name):
    x, y = Test[np.random.randint(0, len(Test))]
    x.unsqueeze_(0)
    y.unsqueeze_(0)
    x, y = x.to(device), y.to(device)
    word_idx = x[:, 1:]
    siz = y.shape
    word_embed = model.embedding(word_idx)
    if name == 'lstm':
        Out, (hk, ck) = model.lstm(word_embed)
    else:
        out, hk = model.gru(word_embed)
    #print(Out.shape)
    pred_idx = torch.zeros_like(y)
    w_idx = x[:, 0].unsqueeze(-1)
    #hk, ck = hout, cout
    #print(hk.shape, ck.shape)
    #print(w_idx.shape)
    for k in range(max(Len_Verse)):
        w_embed = model.embedding(w_idx)
        if name == 'lstm':
            outk, (hk, ck) = model.lstm(w_embed, (hk, ck))
        else:
            outk, hk = model.gru(w_embed, hk)
        logitk = model.linear(outk)
        w_idx = torch.argmax(logitk, dim = -1)
        #print('fnjgjf', w_embed.shape)
        pred_idx[:, k] = w_idx.squeeze()
        #print(w_idx.shape)
        #print(amoerfan)
    x.squeeze_()
    y.squeeze_()
    pred_idx.squeeze_()
    first_verse = [Dict[p] for p in x.tolist()]
    true_verse = [Dict[p] for p in y.tolist()]
    pred_verse = [Dict[p] for p in pred_idx.tolist()]
    #print(pred_verse)
    first_verse = [val for val in first_verse if val != '<خالی>']
    true_verse = [val for val in true_verse if val != '<خالی>']
    first_verse = [val for val in first_verse if val != '<شروع>']
    true_verse = [val for val in true_verse if val != '<شروع>']
    first_verse = [val for val in first_verse if val != '.']
    true_verse = [val for val in true_verse if val != '.']
    pred_verse = [val for val in pred_verse if val != '<خالی>']
    pred_verse = [val for val in pred_verse if val != '<شروع>']
    pred_verse = [val for val in pred_verse if val != '.']

    str_first = ''
    str_true = ''
    str_pred = ''
    for string in first_verse:
        str_first += string + ' '
    for string in true_verse:
        str_true += string + ' '
    for string in pred_verse:
        str_pred += string + ' '
    print(str_first, str_true, str_pred, sep = '\t\t')
    #print(str_true)
    #print(str_pred)

We show some predictions of the "LSTM" model on the test dataset. we predict the second stanza when we apply the first one to the network.

In [43]:
# Showing the LSTM model on the "Test dataset"
print('مصرع اول', 'مصرع دوم', 'مصرع تولید شده با شبکه', sep = '\t\t\t\t')
print('\n')
for k in range(10):
    Generate_Verse(model_lstm, 'lstm')
    print('\n')

مصرع اول				مصرع دوم				مصرع تولید شده با شبکه


به آوردگه گشت و نیزه بگاشت 		چو لختی بگردید نیزه بداشت 		که بد مرز توران ترا یار شیر 


بدانست هومان که آمد سوار 		همه گرزور بود و شمشیردار 		سوی چاره ها سوی دستان سوار 


یکی گفت زان فیلسوفان به شاه 		که بر ژرف دریا ترا نیست راه 		گزین کرد زان دانش و رای شاه 


فرستاده گفت ای خردمند مرد 		مگر داند او گرد دانا مگرد 		مرا بیهده نیست این گفت وگوی 


هما راه جستیم و بگریختیم 		به دام بلا بر نیاویختیم 		نه اندر خور کار جایی نه دشت 


چشیده بسی از جهان شور و تلخ 		بیاری گستهم نوذر ببلخ 		اگر روز باشدت نیست ساز 


کزان بوم خیزد سپهبد چوتو 		فزون آفریناد ایزد چو تو 		چنان دان که کشتی ازو بار و یار 


بدو گفت بهرام کای بدکنش 		نزیبد همی بر تو جز سرزنش 		دل و چشم دشمن به خم هوا 


همه پهلوانان روی زمین 		برو یکسره خواندند آفرین 		منوچهر را خواندند آفرین 


چه با مهد زرین به دیبای چین 		بگوهر بیاراسته همچنین 		برو آفرین کرد و پیروز بخت 




We show some predictions of the "GRU" model on the test dataset. we predict the second stanza when we apply the first one to the network.

In [48]:
print('مصرع اول', 'مصرع دوم', 'مصرع تولید شده با شبکه', sep = '\t\t\t\t')
print('\n')
for k in range(10):
    Generate_Verse(model_gru, 'gru')
    print('\n')

مصرع اول				مصرع دوم				مصرع تولید شده با شبکه


چو روی تخوار او فروزان بدید 		از اندوه چندان دلش بردمید 		زمین را ببوسید و شد ناپدید 


به فر تو گفتا همه بهتریست 		ابا تو همه رنج رامشگریست 		بگویند کاین خویشتن را بخوان 


سپه کوه تا کوه صف برکشید 		پی مور شد بر زمین ناپدید 		دمان و دنان برگرفتند روی 


که خود پیش او دم توان زد درشت 		ورا گردش اختر بد بکشت 		بب و دل باز بینند روی 


گر ایدونک گشتاسپ از روی بخت 		نیابد همی سیری از تاج و تخت 		بدو تاج و تخت و نه تاج و کلاه 


فریبرز گفت آنچ خسرو بگفت 		که با جان پاکش خرد باد جفت 		که او را جز ایدر نباید نهفت 


بیامد برخش اندر آورد پای 		کمر بست و پوشید رومی قبای 		به دستوری بازگشتن به جای 


به پیش بزرگان ستایش کنیم 		همان پیش یزدان نیایش کنیم 		نهفته به بازی نیایش کنیم 


به نزدیک یزدان ز تخمی که کشت 		به باید بپاداش خرم بهشت 		که او را نباشد کسی او چه داشت 


هم ان کین هرمز کنم خواستار 		دگرکاندر ایران منم شهریار 		که بر من یکی روزگار آوری 


